In [1]:
%run common.ipynb

In [2]:
# Parameters
# Chose the variables to crate the model
input_cols = ['shape', 'profile_entrance', 'profile_exit', 'rising_entrance', 'rising_exit', 'shards', 'feathering', 'entrance_mounding', 'center_mounding', 'exit_mounding', 'mounding']
result_col = 'result'
train_dataset_file = 'DATA/Train/serration_prediction_train.xlsx'
test_dataset_file = 'DATA/Validation/serration_prediction_validation.xlsx'

In [3]:
# The data are displayed on a table
display_input_data(
    train_dataset_file,
    input_cols
)

,shape_E,shape_indeterminate,profile_entrance_V,profile_exit_V,rising_entrance_bilateral,rising_entrance_single,rising_exit_bilateral,rising_exit_single,shards_present,feathering_present,entrance_mounding_bilateral,entrance_mounding_single,center_mounding_bilateral,center_mounding_single,exit_mounding_bilateral,exit_mounding_single,mounding_marked,mounding_not marked
0,0,1,1,1,0,1,0,1,0,0,0,1,0,0,0,1,0,1
1,0,1,0,1,0,1,1,0,0,0,0,1,0,0,0,1,0,1
2,0,1,0,1,0,1,1,0,0,0,0,1,0,0,1,0,0,1
3,0,1,0,1,0,1,1,0,0,0,0,1,0,0,1,0,0,1
4,0,1,0,1,0,1,1,0,0,0,0,1,0,0,1,0,0,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
275,1,0,1,1,0,1,0,1,0,0,0,1,0,0,0,1,1,0
276,1,0,1,0,1,0,1,0,0,0,0,0,0,0,0,0,0,0
277,1,0,1,1,0,0,0,1,0,0,0,0,0,0,0,0,0,0
278,1,0,1,1,0,0,0,0,0,1,0,0,0,0,0,0,0,0


In [4]:
# The two datasets are tested for compatibility 
check_input_data_indices(train_dataset_file, test_dataset_file, input_cols)

Datasets are compatible


In [5]:
# Logistic regression model
model = train_model(
    train_dataset_file,
    input_cols,
    result_col
)
model

LogisticRegression(random_state=0, solver='liblinear')

In [11]:
# Test for overfitting
# Evaluation of the training
x_train, y_train = prepare_dataset(train_dataset_file, input_cols, result_col)
model.score(x_train, y_train)

0.9

In [7]:
# 11 variables were used 
evaluate_model(
    model,
    test_dataset_file,
    input_cols,
    result_col
)

0.7164179104477612

In [10]:
# Model optimisation, search for the optimal parameter combination
# Printing score best and worst model
run_input_optimizer(train_dataset_file, test_dataset_file, input_cols, result_col)

Testing combinations of length 2
Testing combinations of length 3
Testing combinations of length 4
Testing combinations of length 5
Testing combinations of length 6
Testing combinations of length 7
Testing combinations of length 8
Testing combinations of length 9
Testing combinations of length 10
Testing combinations of length 11
Max score: 0.8805970149253731
Best columns combination: ('shape', 'profile_entrance', 'rising_entrance', 'shards', 'mounding')
Min score: 0.4626865671641791
Worse columns combination: ('feathering', 'entrance_mounding')
